# DQL Query Examples

Before running, be sure to execute the code in the [Setup](#Setup) cell.

## Table of Contents
1. [Setup](#Setup)
1. [Syntax Basics](#syntax-basics)
1. [Query Variables](#query-variables)
1. [Query Variables Continued](#query-variables-continued)
1. [Value Variables](#value-variables)
1. [Examining the UID Map of Value Variables](#examining-the-uid-map-of-value-variables)



## Setup

In [ ]:
import pydgraph
import json

client_stub = pydgraph.DgraphClientStub(addr='localhost:9080', options=[('grpc.max_receive_message_length', 1024*1024*1024)])
client = pydgraph.DgraphClient(client_stub)
print(client.check_version())

## Syntax Basics

In [ ]:
query = """
{
  all_peeps(func: type(Person), first: 10) {
    Person.name
    Person.pets {
      Pet.name
    }
  }
}
"""

res = client.txn(read_only=True).query(query)
print(json.dumps(json.loads(res.json), indent=2))
print("Total time: {0}ms".format(res.latency.total_ns/1000000))

## Query Variables

Variables produce either sets of UIDs (in the case of *variable blocks*) or in the case of *Value Variables*, maps of UIDs to calculated scalars.

The following query is identical to the one above, but illustrates how variable blocks work. The special `uid` function instructs the final output block to consider all UIDs that were collected in the variable block. By convention, variables are normally represented in all-caps.

In [ ]:
query = """
{
  PEOPLE as var(func: type(Person))
  
  my_peeps(func: uid(PEOPLE), first: 10) {
    Person.name
  }
}
"""

res = client.txn(read_only=True).query(query)
print(json.dumps(json.loads(res.json), indent=2))
print("Total time: {0}ms".format(res.latency.total_ns/1000000))

## Query Variables Continued

A slightly more complicated variable in that the PETS UID set is created in the space of the subgraph that is Persons who are older than 80. The PETS variable will have the UIDs of all Pets for those People older than 80.

In [ ]:
query = """
{
    var(func: type(Person)) @filter(gt(Entity.age, 80)) {
        PETS as Person.pets
    }
  
    pets_of_elderly(func: uid(PETS)) {
        Pet.name
        Pet.owner {
            Person.name
            Entity.age
        }
    }
}
"""

res = client.txn(read_only=True).query(query)
print(json.dumps(json.loads(res.json), indent=2))
print("Total time: {0}ms".format(res.latency.total_ns/1000000))

## Value Variables

Unlike variables declared in blocks, *value variables* store computed scalar values in an internal map, the key of this map is the UID encountered during a block's execution. The value is one or more computed scalars.

In [ ]:
query = """
{
    # Find all pets of people under 21 and store the age of the pet in the PET_AGE value variable
    var(func: type(Person)) @filter(le(Entity.age, 21)) {
        Person.pets {
            PET_AGE as Entity.age
        }
    }

    # Simply show the maximum age of the pets found in the first block
    oldest_pet_of_the_young() {
        age: max(val(PET_AGE))
    }
}
"""

res = client.txn(read_only=True).query(query)
print(json.dumps(json.loads(res.json), indent=2))
print("Total time: {0}ms".format(res.latency.total_ns/1000000))

## Examining the UID Map of Value Variables

In the example above, the final output block simply shows the maximum value from the UID map that is internally created in the first block. That UID map would look some thing like:

```
{
    "<UID-OF-PET-1>": 14,
    "<UID-OF-PET-2>": 15,
    "<UID-OF-PET-3">: 4,
    ...
}
```

And the final blick `max(val(PET_AGE))` scans this map for the maximum scalar.

In this query, we dump the "internal map" that Dgraph creates.

In [ ]:
query = """
{
    # Find all pets of people under 21 and store the age of the pet in the PET_AGE value variable
    var(func: type(Person)) @filter(le(Entity.age, 21)) {
        Person.pets {
            PET_AGE as Entity.age
        }
    }

    check_out_the_uid_map(func: uid(PET_AGE), orderdesc: Entity.age) {
        Pet.name
        petAge:Entity.age
        Pet.owner {
            Person.name
            Entity.age
        }
    }
}
"""

res = client.txn(read_only=True).query(query)
print(json.dumps(json.loads(res.json), indent=2))
print("Total time: {0}ms".format(res.latency.total_ns/1000000))